# Universidad del Valle de Guatemala  
## Facultad de Ingeniería  
### Departamento de Ciencias de la Computación  
#### Curso: CC3084 – Data Science  
#### Semestre II – 2025  

# Proyecto 1 – Obtención y Limpieza de Datos

### Autores:
- José Rodrigo Marchena – 22398  
- Andre Yatmian Jo – 22199  
- Héctor Daniel Penedo – 22217  
- Sofía Velásquez – 22049  

---

### Introducción

En este proyecto se trabaja con datos reales del Ministerio de Educación de Guatemala sobre establecimientos educativos a nivel diversificado. El objetivo principal es obtener, consolidar, limpiar y dejar listo este conjunto de datos para posteriores análisis, asegurando que el proceso de limpieza sea transparente, reproducible y bien documentado.

La limpieza se enfocará en eliminar errores comunes como inconsistencias tipográficas, duplicados, valores faltantes y variaciones no intencionales en nombres de instituciones. Las variables clave como nombre del establecimiento, dirección y teléfono recibirán especial atención, ya que son esenciales para identificar correctamente cada unidad educativa.

---

In [448]:
import pandas as pd
# Cargar el archivo CSV creado a partir del script de readfiles.py que unifica todos los xmls de establecimientos en un solo CSV
file_path = "./datos/establecimiento_all.csv"
df = pd.read_csv(file_path)

In [449]:
# Mostrar dimensiones generales
print("Dimensiones iniciales del dataset:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# Visualizar primeras filas
df.head()

Dimensiones iniciales del dataset:
Número de filas: 6153
Número de columnas: 18


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,DEPARTMENT
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ


#### Análisis inicial

El dataset cuenta con **6,153 registros** y **18 columnas**. Cada fila representa un establecimiento educativo en un municipio y departamento específico de Guatemala.  
A continuación, se listan las variables que componen el dataset:

- `CODIGO`: Identificador único del establecimiento
- `DISTRITO`: Distrito escolar
- `DEPARTAMENTO`, `MUNICIPIO`: Ubicación geográfica
- `ESTABLECIMIENTO`: Nombre del establecimiento
- `DIRECCION`: Dirección física
- `TELEFONO`: Número telefónico de contacto
- `SUPERVISOR`, `DIRECTOR`: Autoridades del establecimiento
- `NIVEL`, `SECTOR`, `AREA`, `STATUS`: Clasificaciones administrativas
- `MODALIDAD`, `JORNADA`, `PLAN`: Características del modelo educativo
- `DEPARTAMENTAL`: Información adicional geográfica
- `DEPARTMENT`: Departamento de origen del archivo original

Este conjunto de datos presenta un nivel de **desorganización moderada**, con potenciales inconsistencias de formato, errores ortográficos y posibles duplicados.


In [450]:
# Analizar variables tipo texto con posibles inconsistencias
text_cols = df.select_dtypes(include='object')
variables_sucias = []
for col in text_cols.columns:
    nulls = df[col].isnull().sum()
    uniques = df[col].nunique()
    if nulls > 0 or uniques < len(df) * 0.95:
        variables_sucias.append(col)

variables_sucias


['DISTRITO',
 'DEPARTAMENTO',
 'MUNICIPIO',
 'ESTABLECIMIENTO',
 'DIRECCION',
 'TELEFONO',
 'SUPERVISOR',
 'DIRECTOR',
 'NIVEL',
 'SECTOR',
 'AREA',
 'STATUS',
 'MODALIDAD',
 'JORNADA',
 'PLAN',
 'DEPARTAMENTAL',
 'DEPARTMENT']

#### Variables con posibles problemas de limpieza:

Se identificaron las siguientes variables con alto potencial de requerir limpieza debido a valores faltantes, inconsistencias de escritura, o cardinalidad irregular:

- `DISTRITO`, `DEPARTAMENTO`, `MUNICIPIO`
- `ESTABLECIMIENTO`, `DIRECCION`, `TELEFONO`
- `SUPERVISOR`, `DIRECTOR`
- `NIVEL`, `SECTOR`, `AREA`, `STATUS`
- `MODALIDAD`, `JORNADA`, `PLAN`
- `DEPARTAMENTAL`, `DEPARTMENT`

Estas variables probablemente presentan duplicados, uso inconsistente de mayúsculas/minúsculas, errores de ortografía, símbolos extraños o valores nulos.


### Estrategias para la limpieza de datos

Siguiendo con las variables con posibles problemas de limpieza, se propone para cada una de ellas una o varias estrategias que permitan reducir la variación y mejorar los resultados del análisis.

<ol>
    <li>DISTRITO</li>
        Se plantea:<br>
        <ul>
            <li>Variable categórica que agrupa a los establecimientos educativos dentro de un mismo municipio. Los primeros dos dígitos indican el departamento y los últimos tres identifican al sector. Se verificará que sean cadenas de texto que contengan solo números y el caracter "-" separando ambas partes.</li>
            <li>Descartar valores faltantes o incorrectos.</li>
        </ul>
    <li>DEPARTAMENTO</li>
        Se plantea:<br>
        <ul>
            <li>Se deberá verificar que todas las instancias estén escritas en <b><u>mayúsculas</u></b> y que sea un nombre existente.</li>
        </ul>
    <li>MUNICIPIO</li>
        Se plantea:<br>
        <ul>
            <li>Al igual que con DEPARTAMENTO, se deberá verificar que todas las instancias estén escritas en <b><u>mayúsculas</u></b> y que el nombre de municipio exista.</li>
        </ul>
    <li>ESTABLECIMIENTO</li>
        Se plantea:<br>
        <ul>
            <li>Varias de las entradas para esta columna contienen palabras encerradas en comillas ("), por lo que se propone eliminarlas para mejorar la legibilidad. También se deberá verificar que todos los nombres estén escritos en <b><u>mayúsculas</b></u>.</li>
        </ul>
    <li>DIRECCION</li>
        Se plantea:<br>
        <ul>
            <li>La primera propuesta para esta variable sería, en caso de que presente demasiada variabilidad, eliminarla. Algunos establecimientos presentan direcciones únicas, por lo que no aportan demasiada información. Además, podría resultar más conveniente simplemente filtrar por municpio y departamento.</li>
            <li>En caso de determinar que presenta información relevante, y al igual que con la variable ESTABLECIMIENTO, algunas de las direcciones ingresadas presentan comillas ("), por lo que también se eliminarán. Se podría extender las abreviaciones comunes, por ejemplo:</li><pre>
            - AVE -> AVENIDA
            - Z -> ZONA
            - COL -> COLONIA</pre>
        </ul>
    <li>TELEFONO</li>
        Se plantea:<br>
        <ul>
            <li>Eliminar caracteres no numéricos, validar que todos los números de teléfono contengan únicamente 8 dígitos y separar o eliminar el extra en caso de que haya varios teléfonos en el mismo campo.</li>
        </ul>
    <li>SUPERVISOR y DIRECTOR</li>
        Se plantea:<br>
        <ul>
            <li>Ya sea eliminar u homogeneizar los títulos que aparezcan en ambas variables. Es decir, al igual que en dirección, se deberá expandir las abreviaciones comunes, por ejemplo:</li><pre>
            - LIC -> LICENCIADO
            - DR -> DOCTOR
            - PROF -> PROFESOR</pre>
            <li>Verificar que todas las instancias estén escritas en <b><u>mayúsculas</u></b> y eliminar comillas (") y espacios innecesarios.</li>
        </ul>
    <li>NIVEL</li>
        Se plantea:<br>
        <ul>
            <li>Debido a que el conjunto de datos se enfoca en el nivel diversificado, todos los registros presentan el atributo DIVERSIFICADO en esta variable. Y ya que no presenta ninguna información relevante, se propone su eliminación.</li>
        </ul>
    </br> </br> </br>
    <li>SECTOR</li>
        Se plantea:<br>
        <ul>
            <li>Esta variable categórica puede tomar solamente los siguientes 4 valores "PRIVADO", "OFICIAL", "COOPERATIVA" y "MUNICIPAL". Se mantendran estas palabras para que sea mas sencillo identificar su significado. Asimismo, la unica limpieza que se les realizara sera asegurarnos que todas las instancias esten en <b><u>mayusculas</u></b> y esten escritas igual cuando sean la misma opcion.</li>
        </ul>
    <li>AREA</li>
        Se plantea:<br>
        <ul>
            <li>Nuevamente, esta columna indica solamente dos poibilidades: area URBANA y RURAL. Debido a la simpleza de esta categoria, se mantendran las palabras como estan y la limpieza constara de asegurar que estas esten escritas de igual manera y en <b><u>mayuscula</b></u></li>
        </ul>
    <li>STATUS</li>
        Se plantea:<br>
        <ul>
            <li><b><u>Eliminar</b></u> la columna completamente. Debido a que el origen de la data son escuelas que actualmente estan abiertas, todas las 6,153 entradas presentes estan marcadas con estatus ABIERTO. Por tanto, ya que este campo no provee informacion en lo absoluto, se removera de los datos limpiados.</li>
        </ul>
    <li>MODALIDAD</li>
        Se plantea:<br>
        <ul>
            <li>Dado que las unicas opciones de esta columnas son MONOLINGUE y BILINGUE, se mantendran estas palabras y la propuesta de limpieza es asegurar que estas esten escritas adecuadamente y en <b><u>mayúsculas</b></u></li>
        </ul>
    <li>JORNADA</li>
        Se plantea:<br>
        <ul>
            <li>Mantener un estandar de escritura para las posibles opciones (VESPERTINA, MATUTINA, DOBLE y SIN JORNADA), asegurando que esten bien escritas y en mayuscula</li>
        </ul>
    <li>PLAN</li>
        Se plantea:<br>
        <ul>
            <li>Para esta columna, se plantea realizar una simplificacion a las categorias presentes pues algunas son redundantes y otras un poco vagas. A continuacion se muestran las conversiones a realizar de los tipos presentes:<br><pre>
            - DIARIO(REGULAR) -> REGULAR<br>
            - SEMIPRESENCIAL(DOS DIAS A LA SEMANA) -> SEMIPRESENCIAL<br>
            - SEMIPRESENCIAL -> SEMIPRESENCIAL<br>
            - A DISTANCIA (VIRTUAL) -> VIRTUAL<br>
            - A DISTANCIA -> VIRTUAL<br>
            - FIN DE SEMANA -> FIN DE SEMANA<br></pre>
            Asimismo, se asegurara que estas opciones esten correctamente escritas y en mayusculas.
            </li>
        </ul>
    <li>DEPARTAMENTAL y</li>
    <li>DEPARTMENT</li>
        Se plantea:<br>
        <ul>
            <li>Estas dos columnas parecen contribuir cierta redundancia sobre los datos, pues en la mayoria de las observaciones, repiten su contenido. En el resto, varian ligeramente en su especificacion, por ejemplo GUATEMALA y GUATEMALA ORIENTE. De manera que se  <b><u>uniran estas dos columnas </b></u>, manteniendo el registro que tenga mas informacion y asegurando que esten en mayusculas y escritas correctamente</li>
        </ul>
</ol>

Aplicamos para Distrito una expresión regular para descartar valores incorrectos o faltantes

In [451]:
original_count = len(df)

# Filter rows matching pattern
df = df[df['DISTRITO'].str.match(r'^\d{2}-\d{3}$', na=False)]

filtered_count = len(df)
removed_count = original_count - filtered_count
print(f"Rows removed: {removed_count}")

Rows removed: 0


Para el departamento se verificara que sean mayusculas y el nombre pertenece, se encontro además CIUDAD CAPITAL en la cual se sustituira por GUATEMALA

In [452]:
df['DEPARTAMENTO'] = df['DEPARTAMENTO'].str.replace('CIUDAD CAPITAL', 'GUATEMALA', case=False, regex=False)


Para los municipio verificamos todas estén escritas en mayuscula 

In [453]:
# Convert only those values that are not already fully uppercase
df['MUNICIPIO'] = df['MUNICIPIO'].apply(lambda x: x.upper() if pd.notnull(x) and x != x.upper() else x)


En Establecimiento se verifica si tienen doble comillas o una comilla y luego se va a quitar con replace siempre escritos en mayuscula

In [454]:
df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].str.replace('"', '', regex=False).str.replace("'", '', regex=False)


Solo se aplicara la eliminación de doble comillas ya que todas estan escritas en AVE -> AVENIDA, Z -> ZONA COL -> COLONIA

In [455]:
df['DIRECCION'] = df['DIRECCION'].str.replace('"', '', regex=False).str.replace("'", '', regex=False)


In [456]:
def clean_telefono(value):
    # If it's not a number, return None (we'll drop those rows later)
    if not str(value).isdigit():
        return None
    # Keep only the first 8 digits
    cleaned = str(value)[:8]
    return cleaned

# Apply the cleaning function
df['TELEFONO'] = df['TELEFONO'].apply(clean_telefono)

# Drop rows where TELEFONO is None (non-numeric originally)
df = df.dropna(subset=['TELEFONO'])

SI tiene comillas su nombre se elimina

In [457]:
# Remove " and ' from SUPERVISOR and DIRECTOR
df['SUPERVISOR'] = df['SUPERVISOR'].str.replace(r'["\']', '', regex=True)
df['DIRECTOR'] = df['DIRECTOR'].str.replace(r'["\']', '', regex=True)


Todos son diversificados solo se elimina si son Diversificado

In [458]:
df = df.drop(columns=["NIVEL"])


Todos son PRIVADO, OFICIAL, COOPERATIVA o MUNICIPAL, por lo que simplemente aseguramos que estén en mayúsculas.

In [459]:
df['SECTOR'] = df['SECTOR'].str.upper().str.strip()


Todos los registros poseen URBANA, RURAL o SIN ESPECIFICAR, por lo que simplementre aseguramos que estén en mayúsculas.

In [460]:
df['AREA'] = df['AREA'].str.upper().str.strip()
df

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,DEPARTMENT
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO LA INMACULADA,7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218,08-06-0073-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,CENTRO EDUCATIVO FE Y ALEGRIA NO 49,"CANTÓN CENTRO, SALIDA A SANTA LUCIA LA REFORMA",56945380,MAURICIO OSORIO TZUNUX,FELIPE DE JESÚS TZUNÚN ALVARADO,PRIVADO,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN
5219,08-06-0079-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,COLEGIO PRIVADO MIXTO FLOR DE MARÍA,ZONA 4 CALLE REAL AL CEMENTERIO,45665179,MAURICIO OSORIO TZUNUX,CARLOS ENRIQUE CHACAJ CARRILLO,PRIVADO,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN
5220,08-06-0118-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,COLEGIO PRIVADO MIXTO FLOR DE MARÍA,ZONA 4 CALLE REAL AL CEMENTERIO,45665179,MAURICIO OSORIO TZUNUX,CARLOS ENRIQUE CHACAJ CARRILLO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (DOS DÍAS A LA SEMANA),TOTONICAPÁN,TOTONICAPAN
5221,08-07-0112-46,08-021,TOTONICAPAN,SANTA LUCIA LA REFORMA,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADO,COLONIA LA ESPERANZA,30868763,PEDRO JOSE CHAN RALAC,PEDRO JOSE CHAN RALAC,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN


Debido a que no presenta información relevante, se eliminará la columna STATUS completamente.

In [461]:
df = df.drop(columns=["STATUS"])
df

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,SECTOR,AREA,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,DEPARTMENT
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,PRIVADO,URBANA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,PRIVADO,URBANA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO LA INMACULADA,7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,PRIVADO,URBANA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,OFICIAL,URBANA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,OFICIAL,URBANA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218,08-06-0073-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,CENTRO EDUCATIVO FE Y ALEGRIA NO 49,"CANTÓN CENTRO, SALIDA A SANTA LUCIA LA REFORMA",56945380,MAURICIO OSORIO TZUNUX,FELIPE DE JESÚS TZUNÚN ALVARADO,PRIVADO,URBANA,BILINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN
5219,08-06-0079-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,COLEGIO PRIVADO MIXTO FLOR DE MARÍA,ZONA 4 CALLE REAL AL CEMENTERIO,45665179,MAURICIO OSORIO TZUNUX,CARLOS ENRIQUE CHACAJ CARRILLO,PRIVADO,URBANA,BILINGUE,VESPERTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN
5220,08-06-0118-46,08-019,TOTONICAPAN,SANTA MARIA CHIQUIMULA,COLEGIO PRIVADO MIXTO FLOR DE MARÍA,ZONA 4 CALLE REAL AL CEMENTERIO,45665179,MAURICIO OSORIO TZUNUX,CARLOS ENRIQUE CHACAJ CARRILLO,PRIVADO,URBANA,MONOLINGUE,SIN JORNADA,SEMIPRESENCIAL (DOS DÍAS A LA SEMANA),TOTONICAPÁN,TOTONICAPAN
5221,08-07-0112-46,08-021,TOTONICAPAN,SANTA LUCIA LA REFORMA,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADO,COLONIA LA ESPERANZA,30868763,PEDRO JOSE CHAN RALAC,PEDRO JOSE CHAN RALAC,OFICIAL,URBANA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),TOTONICAPÁN,TOTONICAPAN


Todos los registros poseen MONOLINGÜE o BILINGÜE por lo que simplementre aseguramos que estén en mayúsculas.

In [462]:
df['MODALIDAD'] = df['MODALIDAD'].str.upper().str.strip()

Se realizó una investigación adicional y se deicidió eliminar la jornada "INTERMEDIA" en favor de la jornada "VESPERTINA" al ser la primera una jornada entre "MATUTINA" y "NOCTURNA".

In [463]:
df['JORNADA'] = df['JORNADA'].str.upper().str.strip()
df['JORNADA'] = df['JORNADA'].replace([{"INTERMEDIA" : "VESPERTINA"}])
df['JORNADA'].unique()

/tmp/ipykernel_44153/3754898200.py:2: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df['JORNADA'] = df['JORNADA'].replace([{"INTERMEDIA" : "VESPERTINA"}])


array(['MATUTINA', 'VESPERTINA', 'DOBLE', 'NOCTURNA', 'SIN JORNADA',
       'INTERMEDIA'], dtype=object)

Se eliminaron valores redundantes como 'DIARIO(REGULAR)' y se dejaron únicamente las clases superiores.

In [464]:
df['PLAN'] = df['PLAN'].str.upper().str.strip()
df['PLAN'] = df['PLAN'].replace({
    'DIARIO(REGULAR)': 'REGULAR',
    'SEMIPRESENCIAL (DOS DIAS A LA SEMANA)': 'SEMIPRESENCIAL',
    'A DISTANCIA (VIRTUAL)': 'VIRTUAL',
    'A DISTANCIA': 'VIRTUAL',
    'SEMIPRESENCIAL (FIN DE SEMANA)' : 'FIN DE SEMANA',
    'SEMIPRESENCIAL (UN DÍA A LA SEMANA)': 'SEMIPRESENCIAL',
    'SEMIPRESENCIAL (DOS DÍAS A LA SEMANA)': 'SEMIPRESENCIAL',
    'VIRTUAL A DISTANCIA': 'VIRTUAL',
})
df['PLAN'].unique()

array(['REGULAR', 'FIN DE SEMANA', 'VIRTUAL', 'SEMIPRESENCIAL'],
      dtype=object)

Se combinaron las columnas 'DEPARTAMENTAL' y 'DEPARTAMENT', dejando los valores con la myor información. También se eliminaron los registros con 'CIUDAD CAPITAL' por 'GUATEMALA'.

In [465]:
df['DEPARTAMENTAL'] = df['DEPARTAMENTAL'].str.upper().str.strip()
df['DEPARTMENT'] = df['DEPARTMENT'].str.upper().str.strip()

df['DEPARTAMENTO_FINAL'] = df[['DEPARTAMENTAL', 'DEPARTMENT']].apply(
    lambda x: x['DEPARTAMENTAL'] if len(str(x['DEPARTAMENTAL'])) >= len(str(x['DEPARTMENT'])) else x['DEPARTMENT'],
    axis=1
)

df['DEPARTAMENTO_FINAL'] = df['DEPARTAMENTO_FINAL'].str.upper().str.strip()
df['DEPARTAMENTO_FINAL'] = df['DEPARTAMENTO_FINAL'].replace({
    'CIUDAD CAPITAL': 'GUATEMALA',})
df['DEPARTAMENTO_FINAL'].unique()

df = df.drop(columns=['DEPARTAMENTAL', 'DEPARTMENT'])


In [466]:
df.to_csv("establecimientos_limpio.csv", index=False)